In [1]:
import pandas as pd
from google.colab import drive

In [2]:
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/no_outliers_data.parquet'
df = pd.read_parquet(file_path, engine='pyarrow')

Mounted at /content/drive


In [3]:
print(df)

         VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
0               2  2020-01-01 00:41:38   2020-01-01 00:52:49   
1               2  2020-01-01 00:10:18   2020-01-01 00:22:16   
2               2  2020-01-01 01:03:14   2020-01-01 01:29:45   
3               2  2020-01-01 00:04:11   2020-01-01 00:09:48   
4               2  2020-01-01 00:25:52   2020-01-01 00:32:16   
...           ...                  ...                   ...   
1673165         2  2024-10-31 23:06:35   2024-10-31 23:10:08   
1673166         2  2024-10-31 23:33:30   2024-10-31 23:38:59   
1673167         2  2024-10-31 23:22:12   2024-10-31 23:26:19   
1673168         2  2024-10-31 23:27:01   2024-10-31 23:47:22   
1673169         2  2024-10-31 23:21:53   2024-10-31 23:33:24   

        store_and_fwd_flag  RatecodeID  PULocationID  DOLocationID  \
0                        N         1.0           181           228   
1                        N         1.0            25            61   
2                    

In [4]:
dropoff_index = df.columns.get_loc('lpep_dropoff_datetime')
df.insert(
    dropoff_index + 1,
    'trip_duration',
    df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
)

In [5]:
import numpy as np
df['fare_per_mile'] = np.where(
    (df['trip_distance'].isna()) | (df['trip_distance'] == 0),
    0,
    df['fare_amount'] / df['trip_distance']
)

df.insert(
    loc=df.columns.get_loc('trip_duration') + 1,
    column='day_of_week',
    value=df['lpep_pickup_datetime'].dt.day_name()
)

In [6]:
hour_series = df['lpep_pickup_datetime'].dt.hour

day_of_week_index = df.columns.get_loc('day_of_week')
df.insert(
    loc=day_of_week_index + 1,
    column='hour_of_day',
    value=hour_series
)

In [7]:
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)
df['location_pair'] = df['PULocationID'] + '-' + df['DOLocationID']

df['DOLocationID'] = df['DOLocationID'].astype('Int64')
df['PULocationID'] = df['PULocationID'].astype('Int64')
output_path = '/content/drive/My Drive/data.parquet'
df.to_parquet(output_path)

In [8]:
print(df)

         VendorID lpep_pickup_datetime lpep_dropoff_datetime   trip_duration  \
0               2  2020-01-01 00:41:38   2020-01-01 00:52:49 0 days 00:11:11   
1               2  2020-01-01 00:10:18   2020-01-01 00:22:16 0 days 00:11:58   
2               2  2020-01-01 01:03:14   2020-01-01 01:29:45 0 days 00:26:31   
3               2  2020-01-01 00:04:11   2020-01-01 00:09:48 0 days 00:05:37   
4               2  2020-01-01 00:25:52   2020-01-01 00:32:16 0 days 00:06:24   
...           ...                  ...                   ...             ...   
1673165         2  2024-10-31 23:06:35   2024-10-31 23:10:08 0 days 00:03:33   
1673166         2  2024-10-31 23:33:30   2024-10-31 23:38:59 0 days 00:05:29   
1673167         2  2024-10-31 23:22:12   2024-10-31 23:26:19 0 days 00:04:07   
1673168         2  2024-10-31 23:27:01   2024-10-31 23:47:22 0 days 00:20:21   
1673169         2  2024-10-31 23:21:53   2024-10-31 23:33:24 0 days 00:11:31   

        day_of_week  hour_of_day store_